# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. Flávio Martins

### Turno BD2L19 - Grupo 05
<dl>
    <dt>71 horas (34.8%)</dt>
    <dd>ist102802 Fábio Mata</dd>
    <dt>65 horas (31.9%)</dt>
    <dd>ist102848 Diogo Rodrigues</dd>
    <dt>68 horas (33.3%)</dt>
    <dd>ist103392 Nuno Gonçalves</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db
%config SqlMagic.displaylimit = 70

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;

CREATE TABLE customer(
cust_no INTEGER PRIMARY KEY,
name VARCHAR(80) NOT NULL,
email VARCHAR(254) UNIQUE NOT NULL,
phone VARCHAR(15),
address VARCHAR(255)
);

CREATE TABLE orders(
order_no INTEGER PRIMARY KEY,
cust_no INTEGER NOT NULL REFERENCES customer,
date DATE NOT NULL
--order_no must exist in contains
);

CREATE TABLE pay(
order_no INTEGER PRIMARY KEY REFERENCES orders,
cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL
--age must be >=18
);

CREATE TABLE process(
ssn VARCHAR(20) REFERENCES employee,
order_no INTEGER REFERENCES orders,
PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
address VARCHAR PRIMARY KEY,
lat NUMERIC(8, 6) NOT NULL,
long NUMERIC(9, 6) NOT NULL,
UNIQUE(lat, long)
--address must be in warehouse or office but not both
);

CREATE TABLE office(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
ssn VARCHAR(20) REFERENCES employee,
name VARCHAR(200) REFERENCES department,
address VARCHAR(255) REFERENCES workplace,
PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
SKU VARCHAR(25) PRIMARY KEY,
name VARCHAR(200) NOT NULL,
description VARCHAR,
price NUMERIC(10, 2) NOT NULL,
ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
order_no INTEGER REFERENCES orders,
SKU VARCHAR(25) REFERENCES product,
qty INTEGER,
PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
TIN VARCHAR(20) PRIMARY KEY,
name VARCHAR(200),
address VARCHAR(255),
SKU VARCHAR(25) REFERENCES product,
date DATE
);

CREATE TABLE delivery(
address VARCHAR(255) REFERENCES warehouse,
TIN VARCHAR(20) REFERENCES supplier,
PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%%sql --file populate.sql



## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql
-- (RI-1)

ALTER TABLE employee 
ADD CHECK (EXTRACT(YEAR FROM AGE(CURRENT_DATE, bdate)) >= 18);

(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql
-- (RI-2)

CREATE OR REPLACE FUNCTION check_workplace_exclusivity()
RETURNS TRIGGER AS
$$
BEGIN
    IF (SELECT COUNT(*) FROM office WHERE address = NEW.address)
        + (SELECT COUNT(*) FROM warehouse WHERE address = NEW.address) != 1 THEN
        RAISE EXCEPTION 'Address must be in warehouse or office but not both';
    END IF;
    
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

CREATE CONSTRAINT TRIGGER workplace_exclusivity
AFTER INSERT ON workplace
DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_workplace_exclusivity();

CREATE CONSTRAINT TRIGGER workplace_exclusivity
AFTER INSERT OR UPDATE OR DELETE ON office
DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_workplace_exclusivity();

CREATE CONSTRAINT TRIGGER workplace_exclusivity
AFTER INSERT OR UPDATE OR DELETE ON warehouse
DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_workplace_exclusivity();

(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [ ]:
%%sql
-- (RI-3)

CREATE OR REPLACE FUNCTION check_order_in_contains()
RETURNS TRIGGER AS
$$
BEGIN
    IF (SELECT COUNT(*) FROM contains WHERE order_no = NEW.order_no) = 0 THEN
        RAISE EXCEPTION 'Order_no must exist in contains';
    END IF;
    
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

CREATE OR REPLACE FUNCTION check_order_still_in_contains()
RETURNS TRIGGER AS
$$
BEGIN
    IF (SELECT COUNT(*) FROM contains WHERE order_no = OLD.order_no) = 0
        AND EXISTS(SELECT order_no FROM orders WHERE order_no = OLD.order_no) THEN
        RAISE EXCEPTION 'Order_no must exist in contains';
    END IF;
    
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

CREATE CONSTRAINT TRIGGER check_order_in_contains
AFTER INSERT ON orders
DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_order_in_contains();

CREATE CONSTRAINT TRIGGER check_order_still_in_contains
AFTER UPDATE OR DELETE ON contains
DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_order_still_in_contains();

## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql

WITH paid_orders_total AS (
    SELECT
        cust_no,
        order_no,
        SUM(qty * price) AS total
    FROM
        pay
    INNER JOIN contains USING (order_no)
    INNER JOIN product USING (sku)
    GROUP BY order_no
)
SELECT
    c.cust_no,
    c.name
FROM
    paid_orders_total
INNER JOIN customer c USING (cust_no)
GROUP BY c.cust_no
HAVING SUM(total) >= ALL(
     SELECT
        SUM(total)
    FROM
        paid_orders_total
    INNER JOIN customer USING (cust_no)
    GROUP BY cust_no   
);

2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql

SELECT
    name
FROM
    employee e
WHERE NOT EXISTS (
    SELECT DISTINCT date
    FROM orders
    WHERE EXTRACT(YEAR FROM date) = 2022
    EXCEPT
    SELECT date
    FROM process
    INNER JOIN orders USING (order_no)
    WHERE ssn = e.ssn
);

3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql

SELECT
    month,
    COUNT(not_paid_orders.date)
FROM
    (
        SELECT date
        FROM orders
        WHERE EXTRACT(YEAR FROM date) = 2022
            AND order_no NOT IN (SELECT order_no FROM pay)
    ) AS not_paid_orders
RIGHT JOIN generate_series(1, 12) AS month ON (EXTRACT(MONTH FROM date) = month)
GROUP BY month
ORDER BY month;

## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql

DROP VIEW IF EXISTS product_sales;

CREATE VIEW product_sales AS
SELECT
    sku,
    order_no,
    qty,
    qty * price AS total_price,
    EXTRACT(YEAR FROM date) AS year,
    EXTRACT(MONTH FROM date) AS month,
    EXTRACT(DAY FROM date) AS day_of_month,
    EXTRACT(DOW FROM date) AS day_of_week,
    SUBSTRING(address SIMILAR '%____-___ #"%#"' ESCAPE '#') AS city
FROM
    contains
INNER JOIN product USING (sku)
INNER JOIN orders USING (order_no)
INNER JOIN customer USING (cust_no)
WHERE order_no IN (SELECT order_no FROM pay);

## 4. Desenvolvimento de Aplicação

Todos os ficheiros relacionados com a app podem ser encontrados na directoria <code>/web</code>.
Dentro dessa pasta existem 3 categorias de ficheiros, os ficheiros python (<code>app.py</code> e <code>app.cgi</code>),
stylesheets (<code>static/style.css</code>) e as templates html (<code>base.html</code>, <code>index_base.html</code> e as específicas a cada entidade, que estão distribuídas por 4 pastas: <code>customers</code>, <code>orders</code>, <code>products</code> e <code>suppliers</code>). A template <code>base.html</code> contém a sidebar, os blocos de header, content e navigation (butões para trocar de página nas listagens). Apenas o <code>index_base.html</code> implementa o bloco de navigation, pelo que todos os <code>*indexes*</code> das várias secções herdam desta template (visto que as listagens são feitas nos indexes). Todo o routing/API está implementado em <code>app.py</code>.

Árvore da directoria web:  
![Web directory tree](https://media.discordapp.net/attachments/1103256858334076981/1119339603527217353/image.png)

As funcionalidades que a app disponibiliza são as seguintes:
- Secção de Customers: <b>(Perspectiva do Manager)</b>
    - Listar todos os customers;
    - Criar Customer;
    - Apagar Customer;
    - Visualizar Customer e suas orders: <b>(Perspectiva do Customer)</b>
        - Visualizar informação do Customer;
        - Listar todas as suas orders;
        - Criar nova order;
        - Pagar order;
        - Visualizar order:
            - Visualizar informação relativa à order;
            - Preço total;
            - Consultar estado de pagamento e opção de pagar caso não esteja paga;
            - Listagem de todos os produtos que a compõem;
- Secção de Products: <b>(Perspectiva do Manager)</b>
    - Listar todos os products;
    - Criar product;
    - Apagar product;
    - Editar product:
        - Preço;
        - Descrição;
        - Apagar product;
- Secção de Suppliers: <b>(Perspectiva do Manager)</b>
    - Listar todos os suppliers;
    - Criar supplier;
    - Apagar supplier;

## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql

WITH product_by_city_and_date AS (
    SELECT
        sku,
        city,
        EXTRACT(YEAR FROM generate_series) AS year,
        EXTRACT(MONTH FROM generate_series) AS month,
        EXTRACT(DAY FROM generate_series) AS day_of_month,
        EXTRACT(DOW FROM generate_series) AS day_of_week
    FROM generate_series('2022-01-01'::TIMESTAMP, '2022-12-31'::TIMESTAMP, '1 day')
    CROSS JOIN (SELECT DISTINCT sku FROM product_sales) AS sku_list
    CROSS JOIN (SELECT DISTINCT city FROM product_sales) AS city_list
)
SELECT
    sku,
    COALESCE(city::VARCHAR, 'WORLDWIDE') AS city,
    COALESCE(month::VARCHAR, 'WHOLE YEAR') AS month,
    COALESCE(day_of_month::VARCHAR, 'WHOLE MONTH') AS day_of_month,
    COALESCE(day_of_week::VARCHAR, 'ALL WEEK DAYS') AS day_of_week,
    SUM(COALESCE(qty, 0)) AS total_qty,
    SUM(COALESCE(total_price, 0)) AS total_sales
FROM product_sales
RIGHT JOIN product_by_city_and_date USING (sku, city, year, month, day_of_month, day_of_week)
GROUP BY sku, GROUPING SETS ((city), (month), (day_of_month), (day_of_week), ())
ORDER BY sku, city, month::INTEGER, day_of_month::INTEGER, day_of_week::INTEGER;

2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql

WITH days_of_2022 AS (
    SELECT
        EXTRACT(YEAR FROM generate_series) AS year,
        EXTRACT(MONTH FROM generate_series) AS month,
        EXTRACT(DAY FROM generate_series) AS day_of_month,
        EXTRACT(DOW FROM generate_series) AS day_of_week
    FROM generate_series('2022-01-01'::TIMESTAMP, '2022-12-31'::TIMESTAMP, '1 day')
)
SELECT
    COALESCE(month::VARCHAR, 'WHOLE YEAR') AS month,
    COALESCE(day_of_week::VARCHAR, 'WHOLE MONTH') AS day_of_week,
    AVG(daily_sales)::NUMERIC(13,5) AS avg_sales
FROM (
    SELECT
        month AS month,
        day_of_week AS day_of_week,
        day_of_month AS day_of_month,
        SUM(COALESCE(total_price, 0)) AS daily_sales
    FROM product_sales
    RIGHT JOIN days_of_2022 d USING (year, month, day_of_month, day_of_week)
    GROUP BY month, day_of_month, day_of_week
) AS sales_per_day
GROUP BY GROUPING SETS ((month), (day_of_week), ())
ORDER BY month::INTEGER, day_of_week;

## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no  
FROM orders  
JOIN contains USING (order_no)  
JOIN product USING (SKU)  
WHERE price > 50 AND EXTRACT(YEAR FROM date) = 2023;

Decidimos criar um índice *Hash* para <code>EXTRACT(YEAR from date)</code> na relação <code>orders</code>:
- Visto que todos os *JOIN*s (dois a dois) são feitos através de um atributo com índice (*primary keys*), não existe vantagem em criar um índice nos atributos <code>order_no</code> e <code>SKU</code>. Pelo que nos restam dois atributos candidatos: <code>price</code> e <code>date</code>;
- No atributo <code>price</code>, não faz sentido criar um índice visto que o *PG* filtra as linhas por <code>price</code> à medida que vai dando *JOIN* entre a tabela resultante do primeiro *JOIN* e a tabela <code>product</code>;
- No atributo <code>date</code>, faz sentido criar um índice porque o *PG* faz um *Seq Scan* nas <code>orders</code> para filtrar por este atributo. Visto que o *PG* não consegue processar o <code>EXTRACT</code> como uma comparação de range, um índice no atributo <code>date</code> não seria vantajoso, pelo que criámos um indice *Hash* para <code>EXTRACT(YEAR from date)</code> (optámos por um índice *Hash* em vez de *BTree* porque a condição de filtragem é uma igualdade);
- Assim, o *PG* deixa de fazer um *Seq Scan* nas <code>orders</code> e passa a fazer um *Bitmap Heap Scan* nas <code>orders</code> juntamente com um *Bitmap Index Scan* no índice criado.

In [ ]:
%%sql

DROP INDEX IF EXISTS orders_extract_year_idx;
CREATE INDEX orders_extract_year_idx ON orders USING HASH(EXTRACT(YEAR from date));

### 6.2
SELECT order_no, SUM(qty*price)  
FROM contains  
JOIN product USING (SKU)  
WHERE name LIKE 'A%'  
GROUP BY order_no;

Decidimos criar um índice BTree no atributo <code>name</code> da relação <code>product</code>:
- Visto que o *JOIN* é feito através de um atributo com índice (*primary key*), não existe vantagem em criar um índice no atributo <code>SKU</code>. Pelo que nos resta apenas um atributo candidato: <code>name</code>;
- No atributo <code>name</code>, faz sentido criar um índice porque o *PG* faz um *Seq Scan* na tabela <code>product</code> para filtrar por este atributo. Visto que este filtro utiliza *pattern matching*, criámos um índice *BTree* no atributo <code>name</code>;
- Apesar de termos criado este índice, o *PG* não o estava a utilizar por ser demasiado abrangente (abrangia toda a tabela), o que o torna pouco eficiente. Assim sendo, tivemos que indexar apenas os <code>name</code>'s que seguem o padrão <code>'A%'</code>.
- Desta forma, o *PG* deixa de fazer um *Seq Scan* na tabela <code>product</code> e passa a fazer um *Bitmap Heap Scan* na tabela <code>product</code> juntamente com um *Bitmap Index Scan* no índice criado.

In [ ]:
%%sql

DROP INDEX IF EXISTS product_name_idx;
CREATE INDEX product_name_idx ON product(name) WHERE name LIKE 'A%';